# Load Modules

In [24]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2

# Load Source list files

In [26]:
## This is for LMXB dataset
src_list = pd.read_csv('../source_list/lmxb_segmented_list.txt' , header=None , names = ['fname'] , na_values='NaN')['fname'].to_list()
src_list
source = pd.DataFrame()
for s in src_list[:]:
    temp = pd.read_csv('../source_list/lmxb_updated/'+s).iloc[: , 1:-1]
    source = source.append(temp)
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
display(source)


FileNotFoundError: [Errno 2] No such file or directory: '../source_list/cv_old_updated/XOFFSET_'

# Read Chandra data files 

In [5]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['A_NAME'][index]
    df = pd.read_csv('../data/all_data/lmxb_v2_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    n_obs_cur = len(df_select)
    obs_id = ['LMXB_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['LMXB_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('LMXB_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 99/99 [00:06<00:00, 14.79it/s]


,type,SRC_ID,A_NAME,A_RA,A_DEC,A_DATABASE_TABLE,A_OBSERVATORY,chandra_name,ch_ra,ch_dec,xoffset,num_obs_used,B_NAME,B_RA,B_DEC,num_obs
0,BH,BH0027,J1745-2900___________,266.416667,-29.008611,RITTERLMXB______,_____________,2CXO J174540.1-290029,266.41740,-29.00827,0.000000,71,2CXO_J174540.1-290029_,266.417364,-29.008273,142
1,NS,NS0001,1A_1742-289__________,266.404167,-29.018611,INTREFCAT_______,INTEGRAL_____,2CXO J174536.7-290107,266.40330,-29.01874,0.000000,49,2CXO_J174536.7-290107_,266.403262,-29.018745,97
2,NS,NS0044,XMMU_J004245.2+41172_,10.688333,41.289167,INTREFCAT_______,INTEGRAL_____,2CXO J004246.1+411728,10.69217,41.29133,0.003074,65,2CXO_J004245.2+411722_,10.688401,41.289518,95
3,NS,NS0046,XMMU_J004245.2+41172_,10.688333,41.289167,INTREFCAT_______,INTEGRAL_____,2CXO J004246.1+411728,10.69217,41.29133,0.003074,65,2CXO_J004246.1+411728_,10.692170,41.291326,65
4,NS,NS0057,J1748-2446___________,267.020000,-24.780278,RITTERLMXB______,_____________,2CXO J174804.5-244642,267.01900,-24.77838,0.000000,17,2CXO_J174804.5-244642_,267.019036,-24.778378,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NS,NS0058,J1748-3607___________,267.054583,-36.132500,RITTERLMXB______,_____________,2CXO J174813.1-360758,267.05470,-36.13281,0.000000,1,2CXO_J174813.1-360758_,267.054676,-36.132810,1
95,NS,NS0012,4U_1708-40___________,258.099167,-40.842778,INTREFCAT_______,INTEGRAL_____,2CXO J171223.7-405034,258.09920,-40.84282,0.000000,1,2CXO_J171223.7-405034_,258.099163,-40.842823,1
96,NS,NS0060,J1749-2919___________,267.480833,-29.322222,RITTERLMXB______,_____________,2CXO J174955.3-291919,267.48070,-29.32209,0.000000,1,2CXO_J174955.3-291919_,267.480748,-29.322086,1
97,NS,NS0061,J1751-3037___________,267.806250,-30.623056,RITTERLMXB______,_____________,2CXO J175113.6-303726,267.80690,-30.62403,0.000000,1,2CXO_J175113.6-303726_,267.806945,-30.624030,1


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_0,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,16597,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_1,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,5954,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_2,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6113,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_3,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6363,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_4,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6639,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_94_obs_0,LMXB_94,1,0.0,ACIS,acisfJ1748144m360743_001,2CXO J174813.1-360758,267.0547,-36.13281,3509,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
LMXB_95_obs_0,LMXB_95,1,0.0,ACIS,acisfJ1712270m405010_001,2CXO J171223.7-405034,258.0992,-40.84282,661,1,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
LMXB_96_obs_0,LMXB_96,1,0.0,ACIS,acisfJ1749552m291755_001,2CXO J174955.3-291919,267.4807,-29.32209,7160,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


## Save raw table

In [6]:
df_combined.replace(np.nan , '')
df_combined.to_csv('lmxb_raw_data.csv' ,)

## Filter flags

In [9]:
from features import col_to_drop

In [11]:
df = pd.read_csv('lmxb_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
display(df)
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
#display(df)
df = df.drop(columns=col_to_drop)
df

BEFORE FILTERING


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_0,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,16597,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_1,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,5954,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_2,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6113,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_3,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6363,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
LMXB_0_obs_4,LMXB_0,71,0.0,ACIS,acisfJ1745401m290028_001,2CXO J174540.1-290029,266.4174,-29.00827,6639,0,...,2.261e-06,3.0,0.151,1.991e-05,71,0,0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_94_obs_0,LMXB_94,1,0.0,ACIS,acisfJ1748144m360743_001,2CXO J174813.1-360758,267.0547,-36.13281,3509,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
LMXB_95_obs_0,LMXB_95,1,0.0,ACIS,acisfJ1712270m405010_001,2CXO J171223.7-405034,258.0992,-40.84282,661,1,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
LMXB_96_obs_0,LMXB_96,1,0.0,ACIS,acisfJ1749552m291755_001,2CXO J174955.3-291919,267.4807,-29.32209,7160,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


AFTER FILTERING


,src_id,num_obs,sepn,name,ra,dec,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_0,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,0.0,0.0,4.333e-07,0.0,...,0.001407,8.0,1.0,4.474e-06,8.0,1.0,2.261e-06,3.0,0.151,1.991e-05
LMXB_0_obs_1,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,0.0,0.0,2.791e-07,0.0,...,0.001407,8.0,1.0,4.474e-06,8.0,1.0,2.261e-06,3.0,0.151,1.991e-05
LMXB_0_obs_2,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,0.0,0.0,1.089e-06,0.0,...,0.001407,8.0,1.0,4.474e-06,8.0,1.0,2.261e-06,3.0,0.151,1.991e-05
LMXB_0_obs_3,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,0.0,0.0,1.544e-07,0.0,...,0.001407,8.0,1.0,4.474e-06,8.0,1.0,2.261e-06,3.0,0.151,1.991e-05
LMXB_0_obs_4,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,0.0,0.0,1.0779999999999999e-06,0.0,...,0.001407,8.0,1.0,4.474e-06,8.0,1.0,2.261e-06,3.0,0.151,1.991e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_91_obs_0,LMXB_91,1,0.0,2CXO J173547.0-302858,263.9459,-30.48282,3.413e-07,8.532e-08,5.851e-07,5.109e-16,...,8.933999999999999e-09,5.0,0.8,1.257e-07,0.0,0.247,5.991999999999999e-09,NaN,NaN,NaN
LMXB_92_obs_0,LMXB_92,1,0.0,2CXO J092920.1-312303,142.3341,-31.38427,7.313e-07,4.3880000000000004e-07,1.024e-06,1.2529999999999999e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LMXB_94_obs_0,LMXB_94,1,0.0,2CXO J174813.1-360758,267.0547,-36.13281,1.1940000000000001e-06,7.463e-07,1.642e-06,1.709e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
from features import arranged_cols  , id_cols 

In [18]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,streak_src_flag,likelihood_b,likelihood_h,likelihood_m,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_0,0.000000e+00,0.000000e+00,4.333000e-07,0.000000e+00,0.000000e+00,4.913000e-16,False,5686.50,5930.28,34.42,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_1,0.000000e+00,0.000000e+00,2.791000e-07,0.000000e+00,0.000000e+00,3.801000e-16,False,0.48,0.55,NaN,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_2,0.000000e+00,0.000000e+00,1.089000e-06,0.000000e+00,0.000000e+00,1.483000e-15,False,7.36,5.98,NaN,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_3,0.000000e+00,0.000000e+00,1.544000e-07,0.000000e+00,0.000000e+00,2.387000e-16,False,6.29,6.61,NaN,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_4,0.000000e+00,0.000000e+00,1.078000e-06,0.000000e+00,0.000000e+00,1.586000e-15,False,1.70,2.40,NaN,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_91_obs_0,3.413000e-07,8.532000e-08,5.851000e-07,5.109000e-16,1.277000e-16,8.759000e-16,False,110.22,6.25,114.25,...,8.934000e-09,5.0,0.800,1.257000e-07,0.0,0.247,5.992000e-09,NaN,NaN,NaN
LMXB_92_obs_0,7.313000e-07,4.388000e-07,1.024000e-06,1.253000e-15,7.353000e-16,1.743000e-15,False,75.53,26.18,19.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LMXB_94_obs_0,1.194000e-06,7.463000e-07,1.642000e-06,1.709000e-15,1.068000e-15,2.350000e-15,False,50.51,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,ra,dec,livetime,significance,likelihood,photflux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_0,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,16464.4,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_1,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,17852.2,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_2,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,4855.4,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_3,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,29759.7,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_4,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.00827,4490.4,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_91_obs_0,LMXB_91,1,0.0,2CXO J173547.0-302858,263.9459,-30.48282,18855.8,4.89,114.24570,3.413000e-07,...,8.934000e-09,5.0,0.800,1.257000e-07,0.0,0.247,5.992000e-09,NaN,NaN,NaN
LMXB_92_obs_0,LMXB_92,1,0.0,2CXO J092920.1-312303,142.3341,-31.38427,24445.9,3.70,75.53454,7.313000e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LMXB_94_obs_0,LMXB_94,1,0.0,2CXO J174813.1-360758,267.0547,-36.13281,15794.5,3.09,50.50810,1.194000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
flux_cols = ['flux_aper_b' , 'flux_aper_s' , 'flux_aper_m' , 'flux_aper_h' , 'flux_aper_u']
df_flux_filt = df_ok.copy()
for f in flux_cols:
    df_flux_filt = df_flux_filt[df_flux_filt[f] < 1e-12]
df_flux_filt

,src_id,num_obs,sepn,name,ra,dec,livetime,significance,likelihood,photflux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
LMXB_0_obs_1,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.008270,17852.2,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_4,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.008270,4490.4,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_6,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.008270,5060.4,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_9,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.008270,4980.9,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
LMXB_0_obs_13,LMXB_0,71,0.0,2CXO J174540.1-290029,266.4174,-29.008270,5084.9,337.00,462011.40000,0.000000e+00,...,1.407000e-03,8.0,1.000,4.474000e-06,8.0,1.000,2.261000e-06,3.0,0.151,0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_90_obs_0,LMXB_90,1,0.0,2CXO J212314.5-054753,320.8106,-5.798128,17705.1,4.06,132.76700,1.829000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LMXB_91_obs_0,LMXB_91,1,0.0,2CXO J173547.0-302858,263.9459,-30.482820,18855.8,4.89,114.24570,3.413000e-07,...,8.934000e-09,5.0,0.800,1.257000e-07,0.0,0.247,5.992000e-09,NaN,NaN,NaN
LMXB_92_obs_0,LMXB_92,1,0.0,2CXO J092920.1-312303,142.3341,-31.384270,24445.9,3.70,75.53454,7.313000e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_flux_filt.to_csv('lmxb_data.csv')